In [5]:
import pandas as pd

sub_df1 = pd.read_csv('../../data/20240830_164022_submission.csv')
sub_df2 = pd.read_csv('../../data/20240830_165720_submission.csv')

# 加重平均
score1 = 0.9721
score2 = 0.9713

weight1 = score1 / (score1 + score2)
weight2 = score2 / (score1 + score2)

print(weight1, weight2)

pred = sub_df1['target'] * weight1 + sub_df2['target'] * weight2
sub_df1['target'] = pred

sub_df1.to_csv('../../results/008/merge_20240830_164022+20240830_165720.csv', index=False)


0.5002058248430585 0.49979417515694147


### Import